<a href="https://colab.research.google.com/github/shubhpanchal/EDA--HotelAnalysis-AlmabetterCapstoneProject/blob/main/Copy_of_Hotel_Booking_Analysis_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <b> Have you ever wondered when the best time of year to book a hotel room is? Or the optimal length of stay in order to get the best daily rate? What if you wanted to predict whether or not a hotel was likely to receive a disproportionately high number of special requests? This hotel booking dataset can help you explore those questions!

## <b>This data set contains booking information for a city hotel and a resort hotel, and includes information such as when the booking was made, length of stay, the number of adults, children, and/or babies, and the number of available parking spaces, among other things. All personally identifying information has been removed from the data. </b>

## <b> Explore and analyze the data to discover important factors that govern the bookings. </b>

In [1]:
# Importing necessary libraries and data 

# For data manupulation and numeric operations
import numpy as np
import pandas as pd

# Data Vizualisation Libraries
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# For loading the dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# loading our hotel data
original_data = pd.read_csv('/content/drive/MyDrive/Almabetter/Alma Capstone Projects Folder/EDA-Hotel Analysis/Copy of Hotel Bookings.csv')
original_data.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [3]:
# Lets create a copy of original Dataset and we will work on the copy data.
# Its always better to work on the copy dataset to avaoid accidental loss of data.
hotel = original_data.copy()
hotel.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [4]:
# Checking the shape of the data
hotel.shape
# There are 119390 rows and 32 columns

(119390, 32)

## Data Cleaning.
### Checking for duplicate values and dropping them.

In [5]:
# Lets check the shape of the duplicated data
hotel[hotel.duplicated()].shape
# There are 31994 rows that seems to be duplicate

(31994, 32)

In [6]:
# Lets drop the duplicate values from the data
hotel.drop_duplicates(inplace = True)
# Setting 'inplace=True' so that changes happen in the original dataset
hotel.shape
# As we can see the duplicated data has been dropped

(87396, 32)

## Understanding Data

In [7]:
hotel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87396 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           87396 non-null  object 
 1   is_canceled                     87396 non-null  int64  
 2   lead_time                       87396 non-null  int64  
 3   arrival_date_year               87396 non-null  int64  
 4   arrival_date_month              87396 non-null  object 
 5   arrival_date_week_number        87396 non-null  int64  
 6   arrival_date_day_of_month       87396 non-null  int64  
 7   stays_in_weekend_nights         87396 non-null  int64  
 8   stays_in_week_nights            87396 non-null  int64  
 9   adults                          87396 non-null  int64  
 10  children                        87392 non-null  float64
 11  babies                          87396 non-null  int64  
 12  meal                           

From inital data exploration we see that the dataset consists of both numerical and categorical values. There are even some datetime values stored in the form of object category, which we need to change into proper format. Also there are some categorical variables stored in the form of numerical variables.
  The other thing that we see is that there are some missing values. We will explore the data further and treat the columns with missing values appropriately.
* Lets understand what are the columns:-


1.   **ADR**(*Numeric*): Average Daily Rate defined as - Calculated by dividing the sum of all lodging transactions by the total number of staying nights.
2.   **Adults**(*Integer*): Number of adults
3.   **Agent**(*Categorical*): ID of the travel agency that made the bookings.
4.   **ArrivalDateDayOfMonth**(*Integer*): Day of the month of the arrival date.
5.   **ArrivalDateMonth**(*Categorical*): Month of arrival date with 12 categories (January to December).
6.   **ArrivalDateWeekNumber**(*Integer*):Week number of the arrival date.
7.   **ArrivalDateYear**(*Integer*): Year of arrival date.
8. **AssignedRoomType** (*Categorical*): Code for the type of room assigned to the booking. Sometimes the assigned room type differs from the reserved room type due to hotel operation reasons (e.g. overbooking) or by customer request. Code is presented instead of designation for anonymity reasons.
9. **babies** (*Integer*) :Number of babies.
10. **BookingChanges** (*Integer*): Number of changes/amendments made to the booking from the moment the booking was entered on the PMS until the moment of check-in or cancellation.Calculated by adding the number of unique iterations that change some of the booking attributes, namely: persons, arrival date,nights, reserved room type or meal.
11. **Children** (*Integer*): Number of children. Sum of both payable and non-payable children.
12. **Company**(*Categorical*): ID of the company/entity that made the booking or responsible for paying the booking. ID is presented instead of designation for anonymity reasons.
13. **Country**(*Categorical*): Country of origin.
14. **CustomerType**(*Categorical*):Type of booking, assuming one of four
categories:-
*   Contract - when the booking has an allotment or other type of contract associated to it.
*   Group – when the booking is associated to a group
*   Transient – when the booking is not part of a group or contract, and is not associated to other transient booking.
*   Transient-party – when the booking is transient, but is associated to at least other transient booking.
15. **DaysInWaitingList**(*Integer*):Number of days the booking was in the waiting list before it was confirmed to the customer. Calculated by subtracting the date the booking was confirmed to the customer from the date the booking entered on the PMS.
16. **DepositType**(*Categorical*):  Indication on if the customer made a
deposit to guarantee the booking. This variable can assume three categories:-
* No Deposit – no deposit was made.
* Non Refund – a deposit was made in the value of the total stay cost.
* Refundable – a deposit was made with a value under the total cost of stay.
17. **DistributionChannel**(*Categorical*): Booking distribution channel. The term “TA” means “Travel Agents” and “TO” means “Tour Operators”.
18. **IsCanceled**(*Categorical*): Value indicating if the booking was canceled (1) or not (0).
19. **IsRepeatedGuest**(*Categorical*): Value indicating if the booking name was from a repeated guest (1) or not (0).
20. **LeadTime**(*Integer*): Number of days that elapsed between the entering date of the booking into the PMS and the arrival date. Subtraction of the entering date from the arrival date.
21. **MarketSegment**(*Categorical*): Market segment designation. In categories, the term “TA” means “Travel Agents” and “TO” means “Tour Operators”.
22. **Meal**(*Categorical*): Type of meal booked. Categories are presented in standard hospitality meal packages.
* Undefined/SC – no meal package.
* BB – Bed & Breakfast.
* HB – Half board (breakfast and one other meal – usually dinner).
* FB – Full board (breakfast, lunch and dinner)
23. **PreviousBookingsNotCanceled**(*Integer*): Number of previous bookings not cancelled by the customer prior to the current booking.
24. **PreviousCancellations**(*Integer*): Number of previous bookings that were
cancelled by the customer prior to the current booking.
25. **RequiredCardParkingSpaces**(*Integer*): Number of car parking spaces required by the customer.
26. **ReservationStatus**(*Categorical*):Reservation last status, assuming one of three categories:
* Canceled – booking was canceled by the customer.
* Check-Out – customer has checked in but already departed.
* No-Show – customer did not check-in and did inform the hotel of the reason why.
27. **ReservationStatusDate**(*Date*): Date at which the last status was set.
This variable can be used in conjunction with the ReservationStatus to understand when was the booking canceled or when did the customer checked-out of the hotel.
28. **ReservedRoomType**(*Categorical*): Code of room type reserved. Code is
presented instead of designation for anonymity reasons.
29. **StaysInWeekendNights**(*Integer*): Number of weekend nights (Saturday or
Sunday) the guest stayed or booked to stay at the hotel. Calculated by counting the number of weekend nights from the total number of nights.
30. **StaysInWeekNights**(*Integer*): Number of week nights (Monday to Friday the guest stayed or booked to stay at the hotel. Calculated by counting the number of week nights from the total number of nights.
31. **TotalOfSpecialRequests**(*Integer*): Number of special requests made by the customer (e.g. twin bed or high floor). Sum of all special requests.

The information about the columns of the dataset was found from the following link:
[Hotel Dataset Columns Information](https://www.sciencedirect.com/science/article/pii/S2352340918315191#t0040)










In [8]:
# Lets create a column by combining day, month and year column to create a new columns 'arrival_date'
from datetime import datetime as dt
hotel['arrival_date'] = hotel['arrival_date_year'].astype('str') + hotel['arrival_date_month'] + hotel['arrival_date_day_of_month'].astype('str')
hotel['arrival_date'] = hotel['arrival_date'].apply(lambda x : dt.strptime(x,'%Y%B%d'))
hotel.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,arrival_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02,2015-07-01
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02,2015-07-01
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03,2015-07-01


In [9]:
# Lets calculate length of stay by adding stays_in_weekend_nights and stays_in_week_nights, with that we can compute revenue by multiplying revenue to length of stay.
hotel['length_of_stay'] = hotel['stays_in_weekend_nights'] + hotel['stays_in_week_nights']
hotel['revenue'] = hotel['adr'] * hotel['length_of_stay']
hotel.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,arrival_date,length_of_stay,revenue
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,0,Transient,0.0,0,0,Check-Out,2015-07-01,2015-07-01,0,0.0
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,0,Transient,0.0,0,0,Check-Out,2015-07-01,2015-07-01,0,0.0
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,0,Transient,75.0,0,0,Check-Out,2015-07-02,2015-07-01,1,75.0
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,0,Transient,75.0,0,0,Check-Out,2015-07-02,2015-07-01,1,75.0
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,0,Transient,98.0,0,1,Check-Out,2015-07-03,2015-07-01,2,196.0


### Dealing with Missing values

In [11]:
# Checking for Missing Values
pd.concat([hotel.isnull().sum(),round(100*(hotel.isnull().sum()/len(hotel)),2)], axis=1, keys=['Missing Values', '% Missing Values'])

,Missing Values,% Missing Values
hotel,0,0.00
is_canceled,0,0.00
lead_time,0,0.00
arrival_date_year,0,0.00
arrival_date_month,0,0.00
arrival_date_week_number,0,0.00
arrival_date_day_of_month,0,0.00
stays_in_weekend_nights,0,0.00
stays_in_week_nights,0,0.00
adults,0,0.00


We see that column 'company' have really high number of missing values, 94%. As the portion of missing data is too large its better to drop the column. Also there are some other columns that have missing values so lets check those too and treat them accordingly.

In [12]:
# Sorting the null values in a descending manner
hotel.isnull().sum().sort_values(ascending = False)[:10]

company                 82137
agent                   12193
country                   452
children                    4
hotel                       0
reserved_room_type          0
booking_changes             0
deposit_type                0
days_in_waiting_list        0
customer_type               0
dtype: int64

In [14]:
# Dropping the company columns and verifying it
hotel.drop(hotel[['company']], axis=1, inplace=True)
'company' in hotel.columns

False

The 'False' represents that we have successfully dropped the company column.

In [16]:
#  Lets treat the missing values in agent column
hotel['agent'] = hotel['agent'].astype('str') # Changing the datatype as its a categorical data.
hotel['agent'] = hotel['agent'].fillna('0') # Imputing 0 to represent that the bookings were not done through any agent.
print('The missing values in agent columns are : ',hotel['agent'].isnull().sum())
print('The dtype of agent column is : ', hotel['agent'].dtype)

The missing values in agent columns are :  0
The dtype of agent column is :  object


We have successfully dealt with the missing values in agent columns. We imputed '0' to represent that the bookings were not done through any agent.

In [19]:
# Lets treat the missing values in childrens column
hotel['children'] = hotel['children'].fillna(hotel['children'].median())
hotel['children'] = hotel['children'].astype('int')
print('The Missing Values in children column are : ', hotel['children'].isnull().sum())
print('The dtype of children column is : ', hotel['children'].dtype)

The Missing Values in children column are :  0
The dtype of children column is :  int64


We have changed the dtype of children column as float is used to represent fractions and childrens cannot be in fractions.

In [20]:
# Lets treat the missing values in country column
hotel['country'] = hotel['country'].fillna('OTH')
print('The missing values in country column are : ', hotel['country'].isnull().sum())

The missing values in country column are :  0


We have imputed "OTH", representing customers from other countries as we do not know the country of origin of these customers.

In [21]:
# Lets check if we still have any column with missing values
hotel.isnull().sum().sort_values(ascending=False)[:5]

hotel                 0
customer_type         0
reserved_room_type    0
assigned_room_type    0
booking_changes       0
dtype: int64

We have successfully treated all the missing values in all columns

## Changing the column types to appropriate column types.

In [25]:
# Changing the numerical columns to categorical columns
change_to_object =  ['arrival_date_year','arrival_date_week_number','arrival_date_day_of_month','is_repeated_guest','is_canceled']
for col in change_to_object:
  hotel[col] = hotel[col].astype('str')
  print('Successfully changed the ' + col + ' dtype to :',hotel[col].dtype)

Successfully changed the arrival_date_year dtype to : object
Successfully changed the arrival_date_week_number dtype to : object
Successfully changed the arrival_date_day_of_month dtype to : object
Successfully changed the is_repeated_guest dtype to : object
Successfully changed the is_canceled dtype to : object


We have successfully changed the dtypes of the columns and lets verify it.

In [26]:
hotel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87396 entries, 0 to 119389
Data columns (total 34 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   hotel                           87396 non-null  object        
 1   is_canceled                     87396 non-null  object        
 2   lead_time                       87396 non-null  int64         
 3   arrival_date_year               87396 non-null  object        
 4   arrival_date_month              87396 non-null  object        
 5   arrival_date_week_number        87396 non-null  object        
 6   arrival_date_day_of_month       87396 non-null  object        
 7   stays_in_weekend_nights         87396 non-null  int64         
 8   stays_in_week_nights            87396 non-null  int64         
 9   adults                          87396 non-null  int64         
 10  children                        87396 non-null  int64         
 11  b

Yes, we have successfully dealt with missing values and also changed the dtypes of required columns.

## Defining Numerical and Categorical Columns.

In [28]:
print('Numerical Columns :-')
numerical = hotel.describe().columns
print(numerical)
print('_'*75)
print('Categorical Columns:-')
categorical = hotel.describe(include=['object','category']).columns
print(categorical)
print('_'*75)

Numerical Columns :-
Index(['lead_time', 'stays_in_weekend_nights', 'stays_in_week_nights',
       'adults', 'children', 'babies', 'previous_cancellations',
       'previous_bookings_not_canceled', 'booking_changes',
       'days_in_waiting_list', 'adr', 'required_car_parking_spaces',
       'total_of_special_requests', 'length_of_stay', 'revenue'],
      dtype='object')
___________________________________________________________________________
Categorical Columns:-
Index(['hotel', 'is_canceled', 'arrival_date_year', 'arrival_date_month',
       'arrival_date_week_number', 'arrival_date_day_of_month', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'reserved_room_type', 'assigned_room_type',
       'deposit_type', 'agent', 'customer_type', 'reservation_status',
       'reservation_status_date'],
      dtype='object')
___________________________________________________________________________
